In [1]:
from IPython.display import Markdown, Latex
from datasets import load_dataset
import numpy as np
import pandas as pd

In [2]:
methods = [
    "danjacobellis/MUSDB_opus_12kbps",
    "danjacobellis/MUSDB_Stereo_Li_192c_J8_nf8",
    "danjacobellis/MUSDB_encodec_12kbps",
    "danjacobellis/MUSDB_Stereo_Li_48c_J8_nf8",
    "danjacobellis/MUSDB_stable_audio_fp16"
]

dimensionality_reduction = [
    1,
    32/12,
    5,
    32/3,
    64,
]

names = [
    'Opus',
    'WalloC',
    'EnCodec',
    'WalloC',
    'Stable Audio',
]

metrics = [
    'PSNR',
    'SSDR',
    'SRDR',
]

columns = [
    'Name',
    'Dimensionality Reduction',
    'Compression Ratio',
    'Encode',
    'Decode',
] + metrics + ['CDPAM']

In [3]:
data_rows = []

for i_method, method in enumerate(methods):
    ds = load_dataset(method, split='validation')
    row = [names[i_method], dimensionality_reduction[i_method]]
    bps_mean = np.mean(ds['bps'])
    compression_ratio = 16 / bps_mean
    encode_MS_per_second = 2*np.mean(np.array(ds['L'])/np.array(ds['cpu_encode_time'])/1e6)
    decode_MS_per_second = 2*np.mean(np.array(ds['L'])/np.array(ds['cpu_decode_time'])/1e6)
    row.append(compression_ratio)
    row.append(encode_MS_per_second)
    row.append(decode_MS_per_second)
    for metric in metrics:
        mean = np.mean(ds[metric])
        row.append(mean)
    CDPAM_dB = np.median(-10*np.log10(ds['CDPAM']))
    row.append(CDPAM_dB)
    data_rows.append(row)

df = pd.DataFrame(data_rows, columns=columns)

df_rounded = df.round(4)
md_table = df_rounded.to_markdown(index=False)
display(Markdown(md_table))

Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

/tmp/ipykernel_525206/1111162417.py:16: RuntimeWarning: divide by zero encountered in log10
  CDPAM_dB = np.median(-10*np.log10(ds['CDPAM']))


Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

...

Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/23 [00:00<?, ?it/s]

Generating validation split:   0%|          | 0/250 [00:00<?, ? examples/s]

| Name         |   Dimensionality Reduction |   Compression Ratio |   Encode |   Decode |    PSNR |    SSDR |    SRDR |   CDPAM |
|:-------------|---------------------------:|--------------------:|---------:|---------:|--------:|--------:|--------:|--------:|
| Opus         |                     1      |            120.426  |  11.5483 |  91.808  | 38.6752 | 20.5199 |  5.3936 | 42.5932 |
| WalloC       |                     2.6667 |             18.0084 |  29.3051 |   6.9085 | 45.1197 | 32.9227 | 12.6936 | 46.9636 |
| EnCodec      |                     5      |            116.274  |   2.9412 |   3.1309 | 40.7408 | 24.6244 |  7.2678 | 48.891  |
| WalloC       |                    10.6667 |             59.2196 |  41.9312 |   7.5381 | 33.1436 | 27.5395 |  7.4112 | 43.1933 |
| Stable Audio |                    64      |             63.5474 |   0.3284 |   0.3118 | 37.8695 | 19.4883 |  3.523  | 51.9254 |

In [4]:
df_rounded = df.round(10)
column_format = 'l' + 'rr' * len(metrics)
latex_table = df_rounded.to_latex(
    index=True,
    float_format="%.3g",
    column_format=column_format,
    multicolumn=True,
    multicolumn_format='c',
    bold_rows=True
)
print(latex_table)

\begin{tabular}{lrrrrrr}
\toprule
 & Name & Dimensionality Reduction & Compression Ratio & Encode & Decode & PSNR & SSDR & SRDR & CDPAM \\
\midrule
\textbf{0} & Opus & 1 & 120 & 11.5 & 91.8 & 38.7 & 20.5 & 5.39 & 42.6 \\
\textbf{1} & WalloC & 2.67 & 18 & 29.3 & 6.91 & 45.1 & 32.9 & 12.7 & 47 \\
\textbf{2} & EnCodec & 5 & 116 & 2.94 & 3.13 & 40.7 & 24.6 & 7.27 & 48.9 \\
\textbf{3} & WalloC & 10.7 & 59.2 & 41.9 & 7.54 & 33.1 & 27.5 & 7.41 & 43.2 \\
\textbf{4} & Stable Audio & 64 & 63.5 & 0.328 & 0.312 & 37.9 & 19.5 & 3.52 & 51.9 \\
\bottomrule
\end{tabular}

